In [2]:
import tensorflow as tf
from skimage import io
import numpy as np

In [76]:
image_file_path = r"F:\papers\FeaDistribution\spa\stego\0.2\0001.bmp"
img = io.imread(image_file_path)
img = img[1000:1100, 1000:1100, 0:1]
[height, width, channel] = np.shape(img)
# img = np.reshape(img, [height, width, channel])
img_new = np.zeros([2, height, width, channel], dtype=np.float32)
img_new[0] = img
img_new[1] = img

[[[132]
  [127]
  [119]
  ..., 
  [111]
  [114]
  [114]]

 [[134]
  [126]
  [135]
  ..., 
  [105]
  [105]
  [105]]

 [[136]
  [137]
  [141]
  ..., 
  [113]
  [114]
  [112]]

 ..., 
 [[123]
  [130]
  [136]
  ..., 
  [175]
  [192]
  [206]]

 [[127]
  [127]
  [124]
  ..., 
  [197]
  [199]
  [190]]

 [[127]
  [123]
  [123]
  ..., 
  [201]
  [194]
  [185]]]


In [77]:
img_tensor = tf.convert_to_tensor(img_new,
                                 dtype=tf.float32)
print(img_tensor.get_shape())

(2, 100, 100, 1)


In [61]:
downsampling_kernel1 = tf.constant(value=[1, 0, 0, 0],
                                   dtype=tf.float32,
                                   shape=[2, 2, 1, 1],
                                   name="downsampling_kernel1")

downsampling_kernel2 = tf.constant(value=[0, 1, 0, 0],
                                   dtype=tf.float32,
                                   shape=[2, 2, 1, 1],
                                   name="downsampling_kernel2")

downsampling_kernel3 = tf.constant(value=[0, 0, 1, 0],
                                   dtype=tf.float32,
                                   shape=[2, 2, 1, 1],
                                   name="downsampling_kernel3")

downsampling_kernel4 = tf.constant(value=[0, 0, 0, 1],
                                   dtype=tf.float32,
                                   shape=[2, 2, 1, 1],
                                   name="downsampling_kernel4")

In [78]:
x_stride, y_stride = 2, 2
padding = "VALID"
conv1 = tf.nn.conv2d(input=img_tensor,
                     filter=downsampling_kernel1,
                     strides=[1, x_stride, y_stride, 1],
                     padding=padding,
                     name="conv1")
conv2 = tf.nn.conv2d(input=img_tensor,
                     filter=downsampling_kernel2,
                     strides=[1, x_stride, y_stride, 1],
                     padding=padding,
                     name="conv2")
conv3 = tf.nn.conv2d(input=img_tensor,
                     filter=downsampling_kernel3,
                     strides=[1, x_stride, y_stride, 1],
                     padding=padding,
                     name="conv3")
conv4 = tf.nn.conv2d(input=img_tensor,
                     filter=downsampling_kernel4,
                     strides=[1, x_stride, y_stride, 1],
                     padding=padding,
                     name="conv4")
tf.concat([conv1, conv2, conv3, conv4], 3)


<tf.Tensor 'concat_8:0' shape=(2, 50, 50, 4) dtype=float32>

In [79]:
print(conv1.get_shape())

(2, 50, 50, 1)


In [80]:
def down_sampling(input_data, x_stride, y_stride, name, padding="VALID"):
    """
    get m * m(m is 2 in general) down sampling layer
    :param input_data: the input data tensor [batch_size, height, width, channels]
    :param x_stride: stride in X axis
    :param y_stride: stride in Y axis
    :param name: the name of the layer
    :param padding: the padding method, "SAME" | "VALID"
    :return:
        feature_map: 4-D tensor [number, height, width, channel]
    """
    conv1 = tf.nn.conv2d(input=input_data,
                         filter=downsampling_kernel1,
                         strides=[1, x_stride, y_stride, 1],
                         padding=padding,
                         name="conv1")

    conv2 = tf.nn.conv2d(input=input_data,
                         filter=downsampling_kernel1,
                         strides=[1, x_stride, y_stride, 1],
                         padding=padding,
                         name="conv2")

    conv3 = tf.nn.conv2d(input=input_data,
                         filter=downsampling_kernel1,
                         strides=[1, x_stride, y_stride, 1],
                         padding=padding,
                         name="conv3")

    conv4 = tf.nn.conv2d(input=input_data,
                         filter=downsampling_kernel1,
                         strides=[1, x_stride, y_stride, 1],
                         padding=padding,
                         name="conv4")

    conv_concat = tf.concat([conv1, conv2, conv3, conv4], 3, name=name)

    return conv_concat

In [81]:
conv = down_sampling(img_tensor, 2, 2, "conv", padding="VALID")
print(conv.get_shape())

(2, 50, 50, 4)


(2, 50, 50, 4)
[[[[ 132.]
   [ 119.]
   [ 126.]
   ..., 
   [ 128.]
   [ 117.]
   [ 114.]]

  [[ 136.]
   [ 141.]
   [ 144.]
   ..., 
   [ 110.]
   [ 116.]
   [ 114.]]

  [[ 112.]
   [ 122.]
   [ 132.]
   ..., 
   [ 109.]
   [ 110.]
   [ 112.]]

  ..., 
  [[ 128.]
   [ 116.]
   [ 127.]
   ..., 
   [ 224.]
   [ 213.]
   [ 185.]]

  [[ 124.]
   [ 135.]
   [ 123.]
   ..., 
   [ 220.]
   [ 179.]
   [ 184.]]

  [[ 127.]
   [ 124.]
   [ 128.]
   ..., 
   [ 220.]
   [ 190.]
   [ 199.]]]]
